In [1]:
from PIL import Image
from pathlib import Path
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import csv

In [2]:
def get_length_and_width(pixels):
    # Now let's get length and width
    centre_x,centre_y= np.mean(pixels,0)

    line_length=200

    angels = []

    for radians in np.arange(0,np.pi,np.pi/180):
        line_y_start = centre_y + line_length*np.sin(radians)
        line_y_end = centre_y - line_length*np.sin(radians)
        line_x_start = centre_x + line_length*np.cos(radians)
        line_x_end = centre_x - line_length*np.cos(radians)

        line_x = np.linspace(line_x_start,line_x_end,200)
        line_y = np.linspace(line_y_start,line_y_end,200)

        # the image is stored as integers, so the line should be converted to ints
        line = np.concatenate([line_x[np.newaxis,:],line_y[np.newaxis,:]]).T.astype(int)

        angels.append(np.sum([(l==pixels).all(1).any() for l in line]))

    angels = np.array(angels)

    ####
    # Length = Max
    # Width = 90 degree offset of length
    ####
    #
    # length_locations = np.argwhere(angels==np.max(angels))
    # width_locations = np.where(length_locations>90,length_locations-90,length_locations+90)
    #
    # length = np.mean(angels[length_locations])
    # width = np.mean(angels[width_locations])

    
    ####
    # Length = Max
    # Width = Min
    #####
    length = np.max(angels)
    width = np.min(angels)

    return length,width

In [4]:
blob_id = 0

too_big = 4000

blobs = {}

folder_path = '10x'


count = 0

for file_idx, file in enumerate(Path(folder_path).glob('*.png')):
    #running on a selection of images to test
    if "GUNDA_ACC" in file.name:
        count+=1
        pic = Image.open(file)
        print(file.name)
        print(pic)
        # the transposition makes the first axis the x value  So now things are stored in (x,y) pairs
        # pic is a multidimensional array representing an image. 
        # This extracts the third channel(blue intensity), and transposes the resulting array. 
        as_array = np.array(pic)[...,2].T
        image_dims = as_array.shape
        
        bright = np.argwhere(as_array>240)

        pixel_has_been_considered = [[0,0]]

        for idx, seed in enumerate(bright):

            if ((seed == pixel_has_been_considered).all(1)).any():
                continue

            pixels_to_consider = [seed,]

            pixels_in_blob = [seed,]

            while pixels_to_consider:
                if len(pixels_in_blob) > too_big:
                    break
                new_pixels_to_consider = [] 
                for ptc in pixels_to_consider:

                    # This assumes that blobs must be in pixels that touch each other
                    # if you want to allow for for (for example) 2 white spaces that
                    # another part of the blob you could include things like (0,-3), 
                    # (0, 3), etc.
                    #
                    # if you go further than 1 step you should prob include 45 degree
                    # steps like (-1,-1), and (2,2).  As you go further out you should 
                    # include other weird angles like 30 and 60 degrees
                    for change in [(-1,0), (0,-1), (1,0), (0,1)]:

                        # the pixel we are looking at has already been examined
                        if ((ptc+change == pixel_has_been_considered).all(1)).any():
                            continue

                        # if the pixel we ar elooking at is a bright spot
                        if ((ptc+change == bright).all(1)).any():
                            pixels_in_blob.append(list(ptc+change))
                            new_pixels_to_consider.append(ptc+change) # this will be included in the next loop

                        pixel_has_been_considered.append(ptc+change)
                pixels_to_consider = new_pixels_to_consider.copy()

            pixels = np.array(pixels_in_blob)

            if len(pixels_in_blob) > too_big:
                continue

            # exclude circles on the edges.  These are likely to be cut off bits, but either way they will mess
            # with roundess calcs
            if 0 in pixels or (image_dims[0]-1) in pixels[...,0] or (image_dims[1]-1) in pixels[...,1]:
                continue

            length,width = get_length_and_width(pixels)

            # Throw away bad ones
            if width <= 2 or length <= 2:# or blobs[blob_id]['area'] < 10:
                continue

            blobs[blob_id] = {}
            blobs[blob_id]['name'] = file.stem
            blobs[blob_id]['pixels'] = pixels
            blobs[blob_id]['area'] = len(pixels)

            ####
            # edge detection
            ###
            change = [[-1,0], [0,-1], [1,0], [0,1]]
            edge = []
            perimeter = 0
            for p_test in pixels:
                number_of_neighbours = np.sum([np.sum(((p_test+change)==p).all(1)) for p in pixels])
                if number_of_neighbours != 4:
                    edge.append(p_test)    
                    # assume every pixel is a unit square.  The contribution to the overall perimeter is:
                    # 0 if a pixel has 4 neighbours
                    # 1 if it has 3 (i.e. it is on a verticle piece of the hole)
                    # 2 if it has 2 (i.e. its a corner)
                    # 3 if it has 1 (i.e. it is a vertex)

                    perimeter += (4-number_of_neighbours)  

            blobs[blob_id]['edge'] = np.array(edge)
            blobs[blob_id]['perimeter'] = perimeter

            blobs[blob_id]['length'] = length
            blobs[blob_id]['width'] = width

            # 1 = perfect circle, 0 = rubbish
            # 4πA/P^2 from measure of leaf circularity https://www.tandfonline.com/doi/full/10.1080/15592324.2021.1977530
            blobs[blob_id]['roundess'] = (4*np.pi) * len(pixels) / (perimeter)**2
            blob_id += 1

            if (blob_id+1) % 10 == 0:
                print(f"File Idx {file_idx+1} | Point {idx+1} of {len(bright)} | Blobs: {blob_id+1}")


print("count: "+str(count))
print(blobs)

GUNDA_ACC_T5_3_10x.png
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x768 at 0x7FE89AA40550>
File Idx 17 | Point 3853 of 33600 | Blobs: 10
File Idx 17 | Point 9197 of 33600 | Blobs: 20
File Idx 17 | Point 17106 of 33600 | Blobs: 30
File Idx 17 | Point 23881 of 33600 | Blobs: 40
File Idx 17 | Point 29353 of 33600 | Blobs: 50
GUNDA_ACC_T5_2_10x.png
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1024x768 at 0x7FE88820CAC0>
File Idx 31 | Point 1442 of 44356 | Blobs: 60
File Idx 31 | Point 7386 of 44356 | Blobs: 70
File Idx 31 | Point 11447 of 44356 | Blobs: 80
File Idx 31 | Point 15570 of 44356 | Blobs: 90
File Idx 31 | Point 21021 of 44356 | Blobs: 100
File Idx 31 | Point 26050 of 44356 | Blobs: 110
File Idx 31 | Point 32863 of 44356 | Blobs: 120
File Idx 31 | Point 41538 of 44356 | Blobs: 130
GUNDA_ACC_T4_1_10x.png
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1024x768 at 0x7FE88820C0A0>
File Idx 41 | Point 4002 of 56584 | Blobs: 140
File Idx 41 | Point 9579 of 

KeyboardInterrupt: 

In [ ]:
blobs_df = pd.DataFrame(blobs, index=[0])

In [ ]:
blobs_df.to_csv('vessel_data_GUNDA_ACC.csv', index=False)